# Fine-Tuning TinyLLaMA with LoRA

In [ ]:
pip install transformers peft datasets accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

#### Load Model & Tokenizer

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model,prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling


#model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # QLoRA optimization, memory efficient
    device_map="auto"
)


# Prepare model for training
model = prepare_model_for_kbit_training(model)


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

apply Low-Rank Adaptation to freezes most of the model and train only tiny trainable adapters added inside attention layers

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,             # adapter rank
    lora_alpha=32,    # scaling factor
    target_modules=["q_proj", "v_proj"],  # LLaMA attention layers modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


prepare training dataset, here i would like to finetune the model with SQL specifc dataset

In [3]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("goendalf666/sql-chat-instructions")
dataset = ds["train"]
print(dataset[0])


{'training_input': '###context:CREATE TABLE head (age INTEGER)\n    ###human: How many heads of the departments are older than 56 ?\n    ###assistant: SELECT COUNT(*) FROM head WHERE age > 56'}


Tokenize

In [4]:
def format_dataset(x):
    return {"text": x["training_input"]}

dataset = dataset.map(format_dataset)



def tokenize_func(x):
    return tokenizer(x["text"], truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_func, batched=True)


Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

training

In [9]:
import time
start_time = time.time()


In [ ]:


training_args = TrainingArguments(
    output_dir="./tinyllama-sql-lora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    fp16=True,                 # half precision for speed/memory
    save_strategy="epoch",
    #save_strategy="steps",
    #save_steps=500,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()
#trainer.train(resume_from_checkpoint=True)
#trainer.train(resume_from_checkpoint="./tinyllama-sql-lora/checkpoint-500")



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quanti

Step,Training Loss
50,1.006100
100,0.962900
150,0.968100
200,0.964100
250,0.927500
300,0.941400
350,0.902500
400,0.919400
450,0.903400
500,0.864600


In [7]:
end_time = time.time()

elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)

print(f"Fine-tuning completed in: {hours}h {minutes}m {seconds}s")

Fine-tuning completed in: 0h 0m 52s


In [8]:
!zip -r tinyllama-sql-lora.zip tinyllama-sql-lora
from google.colab import files
files.download('tinyllama-sql-lora.zip')


  adding: tinyllama-sql-lora/ (stored 0%)
  adding: tinyllama-sql-lora/checkpoint-100/ (stored 0%)
  adding: tinyllama-sql-lora/checkpoint-100/tokenizer.model (deflated 55%)
  adding: tinyllama-sql-lora/checkpoint-100/scaler.pt (deflated 60%)
  adding: tinyllama-sql-lora/checkpoint-100/adapter_config.json (deflated 55%)
  adding: tinyllama-sql-lora/checkpoint-100/special_tokens_map.json (deflated 73%)
  adding: tinyllama-sql-lora/checkpoint-100/trainer_state.json (deflated 58%)
  adding: tinyllama-sql-lora/checkpoint-100/optimizer.pt (deflated 8%)
  adding: tinyllama-sql-lora/checkpoint-100/tokenizer.json (deflated 85%)
  adding: tinyllama-sql-lora/checkpoint-100/scheduler.pt (deflated 56%)
  adding: tinyllama-sql-lora/checkpoint-100/adapter_model.safetensors (deflated 8%)
  adding: tinyllama-sql-lora/checkpoint-100/rng_state.pth (deflated 25%)
  adding: tinyllama-sql-lora/checkpoint-100/tokenizer_config.json (deflated 69%)
  adding: tinyllama-sql-lora/checkpoint-100/chat_template.jinj

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>